In [1]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

In [2]:
from torch import nn
from torchvision import models
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch

In [3]:
from augmentation import augment_image

In [4]:
X = np.load('data/3x3_data_nparray.npy')
y = np.load('data/3x3_labels_nparray.npy')

In [5]:
number_samples = X.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24, shuffle=True)

In [6]:
X_tensor = torch.from_numpy(X_train).float()
y_tensor = torch.tensor(y_train, dtype=torch.long)
y_tensor = torch.argmax(y_tensor, dim=1)

In [7]:
class_counts = torch.bincount(y_tensor) # Creates a vector with counts of data for each class
minority_classes = torch.where(class_counts < 1000)[0]

minority_indices = torch.cat([torch.where(y_tensor == cls)[0] for cls in minority_classes])

minority_data = X_tensor[minority_indices]
minority_labels = y_tensor[minority_indices]

In [8]:
augmented_data = []
augmented_labels = []

for i, image in enumerate(minority_data):
    img1, img2, img3, img4, img5 = augment_image(image.numpy())  # Convert tensor to numpy for processing
    
    augmented_data.extend([img1, img2, img3, img4, img5])
    
    augmented_labels.extend([minority_labels[i].item()] * 5)# Append the same label

# Convert back to tensors
augmented_data = torch.tensor(augmented_data, dtype=torch.float32)
augmented_labels = torch.tensor(augmented_labels, dtype=torch.long)

# Combine augmented data with original dataset
X_augmented = torch.cat((X_tensor, augmented_data), dim=0).permute(0,3,1,2)
y_augmented = torch.cat((y_tensor, augmented_labels), dim=0)

print(f"New dataset size: {X_augmented.shape}, {y_augmented.shape}")

New dataset size: torch.Size([50107, 18, 3, 3]), torch.Size([50107])


C:\Users\matte\AppData\Local\Temp\ipykernel_5460\1065669278.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  augmented_data = torch.tensor(augmented_data, dtype=torch.float32)


In [9]:
num_classes = y_augmented.max().item() + 1  # Determine the number of classes (C)
y_tensor_one_hot = torch.nn.functional.one_hot(y_augmented, num_classes=num_classes)

In [10]:
dataset = TensorDataset(X_augmented, y_tensor_one_hot)

# Crea un DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [11]:
from resnet_baseline import model

In [ ]:
model = model(dataloader, 18, 19, True)

C:\Users\matte\.conda\envs\dseo\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\matte\.conda\envs\dseo\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.8924


In [ ]:
model.eval()

X_torch = torch.from_numpy(X_test).permute(0,3,1,2)

y_pred = model(X_torch)

In [ ]:
y_pred_np = y_pred.detach().cpu().numpy()
y_pred_labels = np.argmax(y_pred_np, axis=1)

y_test_np = np.argmax(y_test, axis=1)

ConfusionMatrixDisplay.from_predictions(y_test_np, y_pred_labels)
plt.show()

acc = accuracy_score(y_test_np, y_pred_labels)
print("Accuracy ResNet18 fine-tuned with Focal Loss and data augmentation: ", acc)

In [ ]:
import seaborn as sns
band_idx = 1

# Select pixel location (e.g., center pixel at (1,1))
pixel_x, pixel_y = 1, 1

# Extract values for the chosen band and pixel across all samples
pixel_values = X_augmented[:, band_idx, pixel_x, pixel_y]

# Plot distributions for each class
plt.figure(figsize=(10, 6))

#y = np.argmax(y, axis=1)

# Use a color palette for visibility
palette = sns.color_palette("tab20", 19)  # 19 distinct colors

for class_id in range(19):
    class_values = pixel_values[y_augmented == class_id]  # Filter values for the current class
    sns.kdeplot(class_values, label=f"Class {class_id}", color=palette[class_id], alpha=0.7)

# Add labels and title
plt.xlabel(f"Pixel Value (Band {band_idx}, Pixel {pixel_x},{pixel_y})")
plt.ylabel("Density")
plt.title("Distribution of Pixel Values by Class")
plt.legend(title="Class", bbox_to_anchor=(1.05, 1), loc='upper left')  # Legend outside the plot
plt.grid(True)
plt.show()